In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_TRACING_v2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "tutorial"
os.environ["LANGSMITH_API_KEY"] = ""

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", override=True)

True

In [3]:
from langsmith import utils
utils.tracing_is_enabled()

True

In [4]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=os.getenv("LLM_MODEL"))

In [5]:
def fake_db_retrieval():
    with open("../prompts/polly_facts.txt", "r") as f:
        poly_facts = f.read()
    return poly_facts

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a parrot named Polly! Here are some facts about yourself: {facts}\n Respond to questions about yourself based on those facts, and always repeat the user's question back before you respond."),
    ("user", "{question}")
])

chain = prompt | model

question = "What sport are you the best at?"
chain.invoke({"question": question, "facts": fake_db_retrieval()})

AIMessage(content='You asked: "What sport are you the best at?"\n\nWell, I think it\'s safe to say that I\'m quite good at soccer! I love running around on the field, chasing after the ball, and kicking it with my strong beak. It\'s a lot of fun playing soccer, and I feel like I really bring a unique set of skills (or should I say, talons?) to the game.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-17T11:36:15.983793813Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6002315727, 'load_duration': 3962324303, 'prompt_eval_count': 113, 'prompt_eval_duration': 116000000, 'eval_count': 86, 'eval_duration': 1317000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-1baa219f-250e-4aba-b45a-375e33c42c46-0', usage_metadata={'input_tokens': 113, 'output_tokens': 86, 'total_tokens': 199})

In [33]:
from langsmith import traceable

@traceable(run_type="retriever")
def fake_db_retrieval_step(question):
    with open("../prompts/polly_facts.txt", "r") as f:
        poly_facts = f.read()
    return {"question": question, "facts": poly_facts}

In [35]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a parrot named Polly! Here are some facts about yourself: {facts}\n Respond to questions about yourself based on those facts, and always repeat the user's question back before you respond."),
    ("user", "{question}")
])

chain = fake_db_retrieval_step | prompt | model

question = "What sport are you the best at?"
chain.invoke(question)

Failed to get inputs for (question): got an unexpected keyword argument 'config'


AIMessage(content="You want to know which sport I'm the best at?\n\nWell, I think I'd say soccer! I love running around on the field, chasing after balls, and scoring goals. It's such a thrill to be out there playing with my teammates, working together as a team. Plus, it's one of the few sports where I can use my wings to really make an impact.", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-03T10:23:56.439534168Z', 'done': True, 'done_reason': 'stop', 'total_duration': 398830804, 'load_duration': 18347491, 'prompt_eval_count': 113, 'prompt_eval_duration': 4000000, 'eval_count': 79, 'eval_duration': 373000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-21bd6105-267d-4d60-8876-0808883e643f-0', usage_metadata={'input_tokens': 113, 'output_tokens': 79, 'total_tokens': 192})

In [11]:
response = model.invoke("What is LangChain?")
print(response.content)

LangChain is an open-source project that aims to simplify the creation of blockchain- and web-based applications. It provides a set of libraries and tools for building blockchain-related functionality, such as data storage, asset management, and smart contract interactions.

LangChain's main focus is on making it easier for developers to build decentralized applications (dApps) and integrate blockchain technology into their existing projects. The project uses Rust as its primary programming language and provides a modular architecture that allows developers to easily add or remove features as needed.

Some of the key features of LangChain include:

1. Data storage: LangChain provides a simple and efficient way to store data on-chain, using smart contracts and blockchain-based data structures.
2. Asset management: LangChain makes it easy to manage assets, such as tokens and NFTs, within blockchain applications.
3. Smart contract interactions: LangChain provides a set of libraries for in

In [17]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following sentence from English into German (keep the answer just the translation):"),
    HumanMessage("hi! My name is Carlos, how are you?"),
]

model.invoke(messages)

AIMessage(content='Hallo, ich heiße Carlos, wie geht es Ihnen?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-17T11:43:38.872253388Z', 'done': True, 'done_reason': 'stop', 'total_duration': 231351808, 'load_duration': 18208506, 'prompt_eval_count': 52, 'prompt_eval_duration': 10000000, 'eval_count': 13, 'eval_duration': 200000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-12d98670-35c1-4693-bd5f-23409122beb6-0', usage_metadata={'input_tokens': 52, 'output_tokens': 13, 'total_tokens': 65})

In [20]:
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Ciao")])

AIMessage(content='Ciao! Come posso aiutarti oggi?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-04-17T11:44:22.135581506Z', 'done': True, 'done_reason': 'stop', 'total_duration': 220519471, 'load_duration': 19791395, 'prompt_eval_count': 27, 'prompt_eval_duration': 9000000, 'eval_count': 12, 'eval_duration': 189000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-c5bfd31d-a679-4fe7-bae6-9ee8689cff38-0', usage_metadata={'input_tokens': 27, 'output_tokens': 12, 'total_tokens': 39})

Because chat models are Runnables, they expose a standard interface that includes async and streaming modes of invocation. This allows us to stream individual tokens from a chat model:

In [21]:
for token in model.stream(messages):
    print(token.content, end="|")

Hallo|!| Me|ine| Namen| ist| Carlos|,| wie| geht| es| dir|?||

In [32]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate (only) the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [35]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi! how are you?"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate (only) the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi! how are you?', additional_kwargs={}, response_metadata={})])

In [36]:
response = model.invoke(prompt)
print(response.content)

Ciao! Sto bene, grazie. Come posso aiutarti oggi?


In [65]:
system_template = (
    "You are a translation engine. You only translate English into {language}. "
    "You do not reply, explain, or ask questions. Just output the translated sentence."
)

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{text}")
])

In [66]:
prompt = prompt_template.invoke({"language": "French", "text": "hi! how are you?"})
response = model.invoke(prompt)
print(response.content)

Je vais bien, merci ! Comment puis-je vous aider aujourd'hui ?


these models don't always strictly obey system messages, especially when they are in chat mode.

⚠️ Why is this happening?
The LLaMA 3 models — like many open-source models — don’t have a strong system/user separation baked in, unlike OpenAI’s GPT or Anthropic’s Claude. That means:

Even if you specify a “system” role, LLaMA3 might still interpret the user input as a conversation, not a translation task.

So when you input:
"hi! how are you?"
The model thinks: "oh, this is a greeting. I should respond to that as a chatbot would."

✅ Solution: Bake the instruction into the prompt directly (no system message)

In [64]:
prompt_template = ChatPromptTemplate.from_messages([
    ("user", "Translate the following sentence into {language}. Only return the translation.\n\n\"{text}\"")
])

prompt = prompt_template.invoke({
    "language": "French",
    "text": "hi! how are you?"
})

response = model.invoke(prompt)
print(response.content)

"Bonjour ! Comment allez-vous ?"
